In [1]:
# Import the packages
import pandas as pd
from numpy import repeat
from ydata.dataset import Dataset
from ydata.synthesizers.regular import RegularSynthesizer
from ydata.connectors import GCSConnector
from ydata.connectors.filetype import FileType
from ydata.utils.formats import read_json

/opt/conda/lib/python3.7/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


In [2]:
# Read the data (Only if you want to train a new network)
# Initialize the YData's connector 
token = read_json('gcs_credentials.json')
connector = GCSConnector('ydatasynthetic', keyfile_dict=token)

# Read the data
data = connector.read_file('gs://pipelines_artifacts/telco_anomaly_outputs/data_processed.csv', file_type = FileType.CSV)

/opt/conda/lib/python3.7/site-packages/distributed/client.py:1128: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+---------+
| Package     | client | scheduler | workers |
+-------------+--------+-----------+---------+
| cloudpickle | 2.0.0  | 1.6.0     | None    |
+-------------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [3]:
# Initialize and Load an alerady trained Synthesizer.
network_synth = RegularSynthesizer(filter_outliers=False)
network_synth = network_synth.load('processed_data_trained_synth.pkl')

In [ ]:
# In case you want to train a new network.
#network_synth.fit(data)

INFO: 2022-02-14 18:49:45,337 [SYNTHESIZER] - Calculating metadata
INFO: 2022-02-14 18:49:46,758 Start data preprocessing.


/opt/conda/lib/python3.7/site-packages/sklearn/mixture/_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


INFO: 2022-02-14 18:49:50,532 End data preprocessing.
INFO: 2022-02-14 18:49:51,602 [SYNTHESIZER] - Synthesizer init. Synthesizer: CTGAN
INFO: 2022-02-14 18:49:51,604 [SYNTHESIZER] - Processing the data prior fitting the synthesizer.
INFO: 2022-02-14 18:49:57,117 [SYNTHESIZER] - Training running on a cpu device.


In [13]:
# Create the samples and flag them.
synth_sample = network_synth.sample(5000).to_pandas()
synth_sample.index = repeat(999999, len(synth_sample))

INFO: 2022-02-09 15:13:22,454 [SYNTHESIZER] - Start generating model samples.
INFO: 2022-02-09 15:13:22,456 [SYNTHESIZER] - Current batch_size is set as 1000


In [ ]:
connector.write_file(data= synth_sample, path='gs://pipelines_artifacts/telco_anomaly_outputs/data_sampled.csv', index=False)
